<a href="https://colab.research.google.com/github/azizfath/bdpa_lanjut/blob/master/Responsi%20BDPAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**RESPONSI BIG DATA PREDICTIVE ANALYTICS LANJUT**
##**NAMA: FATHURRAHMAN NUR AZIZ**
##**NIM: 20.11.3694**

##**Klasifikasi dataset Iris dengan Algoritma *Decision Tree Classifier***

##1. Persiapan Spark dan Library yang digunakan

In [1]:
#install findspark and pyspark

!pip install -q findspark
!pip install -q pyspark

In [2]:
#import library

from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import VectorAssembler
from sklearn.metrics import confusion_matrix

In [3]:
#create spark session

spark = SparkSession \
  .builder \
  .getOrCreate()

In [4]:
#create spark context

sc = spark.sparkContext

##2. Import Dataset dan Visualisasi Dataset

In [5]:
#import dataset

path = '/content/Iris.csv'

In [6]:
#load dataset

iris = spark.read.csv(path, header=True, inferSchema=True)

In [7]:
#visualisasi dataset 

iris.show()

+---+-------------+------------+-------------+------------+-----------+
| Id|SepalLengthCm|SepalWidthCm|PetalLengthCm|PetalWidthCm|    Species|
+---+-------------+------------+-------------+------------+-----------+
|  1|          5.1|         3.5|          1.4|         0.2|Iris-setosa|
|  2|          4.9|         3.0|          1.4|         0.2|Iris-setosa|
|  3|          4.7|         3.2|          1.3|         0.2|Iris-setosa|
|  4|          4.6|         3.1|          1.5|         0.2|Iris-setosa|
|  5|          5.0|         3.6|          1.4|         0.2|Iris-setosa|
|  6|          5.4|         3.9|          1.7|         0.4|Iris-setosa|
|  7|          4.6|         3.4|          1.4|         0.3|Iris-setosa|
|  8|          5.0|         3.4|          1.5|         0.2|Iris-setosa|
|  9|          4.4|         2.9|          1.4|         0.2|Iris-setosa|
| 10|          4.9|         3.1|          1.5|         0.1|Iris-setosa|
| 11|          5.4|         3.7|          1.5|         0.2|Iris-

In [8]:
#cek tipe data tiap kolom dataset

iris.printSchema()

root
 |-- Id: integer (nullable = true)
 |-- SepalLengthCm: double (nullable = true)
 |-- SepalWidthCm: double (nullable = true)
 |-- PetalLengthCm: double (nullable = true)
 |-- PetalWidthCm: double (nullable = true)
 |-- Species: string (nullable = true)



##3. Menentukan variabel Independent (X / features) dan Variabel Dependent (Y / target)

###**Dari data tersebut, maka:**

*   variabel independent (X) adalah fitur yaitu : **SepalLengthCm, SepalWidthCm, PetalLengthCm, PetalWidthCm**
*   Variabel dependent (y) adalah target yaitu : **Species**, tetapi perlu kita ubah (transform/encoding) menjadi nilai integer dulu



##4. Transform Data
Mengubah seluruh tipe data string menjadi numerik.

In [9]:
#karena hanya variabel species yang berupa string, maka kita akan lakukan transform pada kolom tersebut

#melihat isi unique dari kolom species

iris.select('species').distinct().show()

+---------------+
|        species|
+---------------+
| Iris-virginica|
|    Iris-setosa|
|Iris-versicolor|
+---------------+



In [10]:
#transform kolom species ke integer

from pyspark.sql.functions import when

iris = iris.withColumn('species_int', when(iris['species'] == 'Iris-virginica',0.0) \
                       .otherwise(when(iris['species'] == 'Iris-setosa',1.0) \
                       .otherwise(when(iris['species'] == 'Iris-versicolor',2.0))))

In [11]:
#melihat hasil transform dengan mencari nilai unique dari kolom baru "species_int"

iris.select('species_int').distinct().show()

+-----------+
|species_int|
+-----------+
|        0.0|
|        1.0|
|        2.0|
+-----------+



In [12]:
#menampilkan dataframe yang telah ditambahkan kolom baru species_int

iris.show()

+---+-------------+------------+-------------+------------+-----------+-----------+
| Id|SepalLengthCm|SepalWidthCm|PetalLengthCm|PetalWidthCm|    Species|species_int|
+---+-------------+------------+-------------+------------+-----------+-----------+
|  1|          5.1|         3.5|          1.4|         0.2|Iris-setosa|        1.0|
|  2|          4.9|         3.0|          1.4|         0.2|Iris-setosa|        1.0|
|  3|          4.7|         3.2|          1.3|         0.2|Iris-setosa|        1.0|
|  4|          4.6|         3.1|          1.5|         0.2|Iris-setosa|        1.0|
|  5|          5.0|         3.6|          1.4|         0.2|Iris-setosa|        1.0|
|  6|          5.4|         3.9|          1.7|         0.4|Iris-setosa|        1.0|
|  7|          4.6|         3.4|          1.4|         0.3|Iris-setosa|        1.0|
|  8|          5.0|         3.4|          1.5|         0.2|Iris-setosa|        1.0|
|  9|          4.4|         2.9|          1.4|         0.2|Iris-setosa|     

In [13]:
#cek tipe data tiap kolom dataframe yang telah ditambahkan kolom baru species_int

iris.printSchema()

root
 |-- Id: integer (nullable = true)
 |-- SepalLengthCm: double (nullable = true)
 |-- SepalWidthCm: double (nullable = true)
 |-- PetalLengthCm: double (nullable = true)
 |-- PetalWidthCm: double (nullable = true)
 |-- Species: string (nullable = true)
 |-- species_int: double (nullable = true)



###Mengkombinasikan semua fitur menjadi satu kolom

In [14]:
#memilih fitur dependent (X) yaitu 'SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm','PetalWidthCm'

kolom = ['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm','PetalWidthCm']
fixed_df = VectorAssembler(inputCols = kolom, outputCol='features')

fixed_df = fixed_df.transform(iris)
fixed_df = fixed_df.select(['features', 'species_int'])
fixed_df.show()

+-----------------+-----------+
|         features|species_int|
+-----------------+-----------+
|[5.1,3.5,1.4,0.2]|        1.0|
|[4.9,3.0,1.4,0.2]|        1.0|
|[4.7,3.2,1.3,0.2]|        1.0|
|[4.6,3.1,1.5,0.2]|        1.0|
|[5.0,3.6,1.4,0.2]|        1.0|
|[5.4,3.9,1.7,0.4]|        1.0|
|[4.6,3.4,1.4,0.3]|        1.0|
|[5.0,3.4,1.5,0.2]|        1.0|
|[4.4,2.9,1.4,0.2]|        1.0|
|[4.9,3.1,1.5,0.1]|        1.0|
|[5.4,3.7,1.5,0.2]|        1.0|
|[4.8,3.4,1.6,0.2]|        1.0|
|[4.8,3.0,1.4,0.1]|        1.0|
|[4.3,3.0,1.1,0.1]|        1.0|
|[5.8,4.0,1.2,0.2]|        1.0|
|[5.7,4.4,1.5,0.4]|        1.0|
|[5.4,3.9,1.3,0.4]|        1.0|
|[5.1,3.5,1.4,0.3]|        1.0|
|[5.7,3.8,1.7,0.3]|        1.0|
|[5.1,3.8,1.5,0.3]|        1.0|
+-----------------+-----------+
only showing top 20 rows



##5. Split Data Train dan Data Test

In [15]:
#split dataset dengan komposisi data train = 0.75 dan data test = 0.25

(data_train, data_test) = fixed_df.randomSplit([0.75, 0.25])

##6. Membuat Model Klasifikasi

In [16]:
#membuat model Decission Tree Classifier

dtc = DecisionTreeClassifier(featuresCol="features", labelCol="species_int")

##7. Training Algoritma

In [17]:
#melakukan training berdasar model yang sudah dibuat

dtc = dtc.fit(data_train)

##8. Melakukan Prediksi

In [18]:
#mencoba prediksi

pred = dtc.transform(data_test)
pred.show(10)

+-----------------+-----------+--------------+-------------+----------+
|         features|species_int| rawPrediction|  probability|prediction|
+-----------------+-----------+--------------+-------------+----------+
|[4.4,2.9,1.4,0.2]|        1.0|[0.0,37.0,0.0]|[0.0,1.0,0.0]|       1.0|
|[4.6,3.1,1.5,0.2]|        1.0|[0.0,37.0,0.0]|[0.0,1.0,0.0]|       1.0|
|[4.6,3.2,1.4,0.2]|        1.0|[0.0,37.0,0.0]|[0.0,1.0,0.0]|       1.0|
|[4.6,3.6,1.0,0.2]|        1.0|[0.0,37.0,0.0]|[0.0,1.0,0.0]|       1.0|
|[4.8,3.4,1.6,0.2]|        1.0|[0.0,37.0,0.0]|[0.0,1.0,0.0]|       1.0|
|[4.9,2.4,3.3,1.0]|        2.0|[0.0,0.0,37.0]|[0.0,0.0,1.0]|       2.0|
|[5.0,3.0,1.6,0.2]|        1.0|[0.0,37.0,0.0]|[0.0,1.0,0.0]|       1.0|
|[5.0,3.2,1.2,0.2]|        1.0|[0.0,37.0,0.0]|[0.0,1.0,0.0]|       1.0|
|[5.0,3.5,1.3,0.3]|        1.0|[0.0,37.0,0.0]|[0.0,1.0,0.0]|       1.0|
|[5.2,2.7,3.9,1.4]|        2.0|[0.0,0.0,37.0]|[0.0,0.0,1.0]|       2.0|
+-----------------+-----------+--------------+-------------+----

##9. Evaluasi Model

In [19]:
#menampilkan Prediction Accuracy dari model yang telah dibuat

evaluator = MulticlassClassificationEvaluator(predictionCol="prediction",labelCol='species_int')
accuracy = evaluator.evaluate(pred)
print("Prediction Accuracy: ", str(accuracy))

Prediction Accuracy:  0.9721164021164022


In [20]:
#Menampilkan Confussion Matrix

y_pred=pred.select("prediction").collect()
y_orig=pred.select("species_int").collect()
cm = confusion_matrix(y_orig, y_pred)
print("Confusion Matrix:")
print(cm)

Confusion Matrix:
[[12  0  0]
 [ 0 13  0]
 [ 1  0 10]]


**Interpretasi Hasil Evaluasi:** 

* Hasil klasifikasi termasuk bagus karena prediksi mencapai : 0.9721164021164022
* Dari Confusion Matrix, dapat disimpulkan klasifikasi termasuk bagus karena hanya menghasilkan 1 nilai false, sedangkan hasil true nya sebanyak 12+13+10 = 35